In [1]:
%pip install -r requirements.txt > /dev/null 2>&1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import requests
import urllib3

In [3]:
# Disable insecure request warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
class ChatClient:
    """A base client for interacting with the chat completion API."""
    def __init__(self, model: str, url: str = "https://typingmind.poyner.lan/v1/chat/completions"):
        self.url = url
        self.model = model

    def send_message(self, message: str) -> str:
        payload = {
            "model": self.model,
            "messages": [
                {"role": "user", "content": message}
            ]
        }
        response = requests.post(self.url, json=payload, verify=False)
        try:
            data = response.json()
            choices = data.get("choices", [])
            if not choices:
                raise ValueError("No choices available in response: " + str(data))
            content = choices[0].get("message", {}).get("content", "")
            return content
        except ValueError as e:
            raise ValueError("Response is not in JSON format: " + response.text) from e

In [5]:
class ThalamusClient(ChatClient):
    """Client for sending analysis requests to the Thalamus model."""
    def __init__(self):
        super().__init__(model="baby-einstein/thalamus:latest")

    def analyze(self, sensor: str, input_type: str, input_data: str, threshold_score: str) -> str:
        inner_message = json.dumps({
            "sensor": sensor,
            "input_type": input_type,
            "input_data": input_data,
            "threshold_score": threshold_score
        })
        return self.send_message(inner_message)

In [6]:
class ACCClient(ChatClient):
    """Client for sending evaluation requests to the ACC model."""
    def __init__(self):
        super().__init__(model="baby-einstein/acc:latest")

    def evaluate(self, thalamus_output: str) -> str:
        return self.send_message(thalamus_output)


In [7]:
# Example usage:
if __name__ == "__main__":
    # Define input parameters (without extra quotes)
    sensor = "camera"
    input_type = "image"
    input_data = "object moving in front of camera"
    threshold_score = ".88"

    # Create a ThalamusClient instance and get its response
    thalamus_client = ThalamusClient()
    thalamus_response = thalamus_client.analyze(sensor, input_type, input_data, threshold_score)
    print("Thalamus response:")
    print(thalamus_response)

    # Create an ACCClient instance and evaluate the Thalamus response
    acc_client = ACCClient()
    acc_response = acc_client.evaluate(thalamus_response)
    print("ACC response:")
    print(acc_response)

Thalamus response:
{"region": "Visual Cortex", "schema": "object_recognition", "perception": "An object moved in front of the camera, which was captured in a digital image format with the 'camera' sensor and processed by 'video analysis'.", "message": "{"sensor": "camera", "input_type": "image", "input_data": "object moving in front of camera", "threshold_score": ".88"}"}
ACC response:
{
  "pass_doubt": true,
  "threshold_score": 0.95,
  "message": "{\"region\": \"Visual Cortex\", \"schema\": \"object_recognition\", \"perception\": \"An object moved in front of the camera, which was captured in a digital image format with the 'camera' sensor and processed by 'video analysis'.\",\"sensor\": \"camera\", \"input_type\": \"image\", \"input_data\": \"object moving in front of camera\", \"threshold_score\": .88}\"",
  "feelings": "Albert feels calm curiosity.",
  "significance": 0.75
}
